# GPT From Scratch

Este repositório irá implementar o GPT 2 do zero, com o código bruto do modelo presente na pasta `src`.

O objetivo será realizar o seguinte fluxo de atividades:
* Leitura do dataset;
* Treina modelo;
* Plotagem de métricas durante treino;
* Processa modelo para gerar textos;
* Avaliação dos textos;
* Plotagem da avaliação;
* Conclusão.

---

# Preparando ambiente

Importando pacotes python para utilização do código

In [ ]:
# Importando pacotes python

from os import mkdir, environ
from os.path import exists

from json import load

Importando pacotes para execução do experimento

In [ ]:
from src.dataload.tokenizer import Tokenizer
from src.gptmodel.model import GPTModel

---

# Configurações para rodar o sistema

Aqui ocorrerá as constantes para o processamento do sistema:

In [ ]:
RESULT_DIR = "results"

if not exists(RESULT_DIR): mkdir(RESULT_DIR)

Diretório de banco de dados:

In [ ]:
DATASET_DIR = "dataset/"

Caminho final dos tokens:

In [ ]:
TOKEN_TEXTS = f"{RESULT_DIR}/tokens.txt"

Caminho do melhor modelo:

In [ ]:
BEST_MODEL_PATH = f"{RESULT_DIR}/best_model.pt"

Configuração do GPT:

In [ ]:
GPT_CONFIGURATION = f"src/gpt_configurations.json"

Estatísticas:

In [ ]:
TRAIN_RATIO = 0.8
BATCH_SIZE = 5

Configuração:

In [ ]:
with open(GPT_CONFIGURATION, 'r') as f:
    CONFIGURATION = load(f)

In [ ]:
CONFIGURATION_KEY = "GPT_CONFIG_124M"

In [ ]:
configuration = CONFIGURATION[CONFIGURATION_KEY]

Ambientes de execução:

In [ ]:
environ['CUDA_LAUNCH_BLOCKING'] = "1"

---

# Leitura do Dataset

Tokenizando todo o conjunto de dados para treinamento e salvando em um arquivo de texto:

In [ ]:
tokenizer = Tokenizer(["utf-8", "latin1"])

In [ ]:
tokenizer.generate_data(DATASET_DIR, TOKEN_TEXTS)

Carregando base de dados:

In [ ]:
with open(TOKEN_TEXTS, 'r') as f:
    contents = f.read()

In [ ]:
train_dataloader, test_dataloader = tokenizer.create_dataloaders(
    contents, 
    TRAIN_RATIO, 
    BATCH_SIZE, 
    configuration["vocab_size"], 
    configuration["vocab_size"]
    )

---

# Treino dos modelos

Esta seção irá ocorrer a instanciação e geração dos modelos:

In [ ]:
model = GPTModel(
    vocab_size=configuration["vocab_size"],
    emb_dim=configuration["emb_dim"],
    context_length=configuration["context_length"],
    drop_rate=configuration["drop_rate"],
    n_heads=configuration["n_heads"],
    n_layers=configuration["n_layers"],
    qkv_bias=configuration["qkv_bias"]
    )

In [ ]:
trained_models, final_metrics = GPTModel._train(
    model=model, 
    train_loader=train_dataloader, 
    val_loader=test_dataloader, 
    device_str="cuda",
    path=BEST_MODEL_PATH)